Download forex data from Oanda V20 API

* https://pypi.org/project/oandapyV20/ : Downloading historical data is limited to 5000 records per request.
* More referece: https://medium.com/@dodgervl/get-free-financial-historical-data-from-oanda-v20-api-in-3-easy-steps-54a485cebcce

In [1]:
# identify local timezone

from datetime import datetime

print(datetime.now().astimezone().tzinfo)

Malay Peninsula Standard Time


In [2]:
import json
import pandas as pd
from dateutil import parser
import pytz
from oandapyV20 import API 
import oandapyV20.endpoints.instruments as instruments

with open('demo_account.json') as f:
    account_keys = json.load(f)

client = API(account_keys['token'])

In [16]:
GRANULARITY = 'M1' # H1 M15 M1 S10
GRANULARITY_SEC = 60
OANDA_START_DATE = '2020-01-01'
OANDA_END_DATE = '2021-01-01'
INSTRUMENT = 'EUR_USD'
LOCAL_TIMEZONE = 'Asia/Singapore' # pytz.all_timezones
OANDA_TIMEZONE = 'US/Eastern'
BAR_COUNT_LIMIT = 5000

OUTPUT_FILE = f'oanda_{INSTRUMENT}_{GRANULARITY}_{OANDA_START_DATE}_{OANDA_END_DATE}.parquet.gz'

OANDA_START_TIME = f'{OANDA_START_DATE} 00:00:00'
OANDA_END_TIME = f'{OANDA_END_DATE} 00:00:00'

# The following timezone convertion is needed becuase `timestamp()` will use local timezone
def _convert_tz_oanda_to_local(dt):
    local_tz = pytz.timezone(LOCAL_TIMEZONE)
    oanda_tz = pytz.timezone(OANDA_TIMEZONE)
    return oanda_tz.localize(dt).astimezone(local_tz)

start_dt = _convert_tz_oanda_to_local(parser.parse(OANDA_START_TIME))
end_dt = _convert_tz_oanda_to_local(parser.parse(OANDA_END_TIME))
start_unix = int(start_dt.timestamp())
end_unix = int(end_dt.timestamp())

# Query data
def _extract_single_candle(c):
    return {
        'time':c['time'],
        'complete': c['complete'],
        'volume':float(c['volume']),
        'bid_o':float(c['bid']['o']),
        'bid_h':float(c['bid']['h']),
        'bid_l':float(c['bid']['l']),
        'bid_c':float(c['bid']['c']),
        'ask_o':float(c['ask']['o']),
        'ask_h':float(c['ask']['h']),
        'ask_l':float(c['ask']['l']),
        'ask_c':float(c['ask']['c']),
    }

step = GRANULARITY_SEC * BAR_COUNT_LIMIT
l_data = []
for start_unix_i in range(start_unix, end_unix, step):
    params = {
        'from': str(start_unix_i),
        'to': str(min(start_unix_i + step, end_unix)),
        'granularity': GRANULARITY,
        'price':'AB', # 'A' for ask, 'B' for bid, 'AB' for both.
    }

    r = instruments.InstrumentsCandles(instrument=INSTRUMENT, params=params)
    data = client.request(r)

    df_i = pd.DataFrame(
        [_extract_single_candle(c) for c in data['candles']]
    )

    l_data.append(df_i)

df = pd.concat(l_data, ignore_index=True)

# exam if there are duplicated timestamps
try:
    df.set_index('time', verify_integrity=True)
except:
    print(f"duplicated time identified.")

df['time'] = pd.to_datetime(df['time'])

print(f"Queried data has {len(df)} rows, {df['time'].nunique()} bars, {(~df['complete']).sum()} incompleted bars, "
      f"from {df['time'].min()} to {df['time'].max()} {OANDA_TIMEZONE}.")

df.to_parquet(OUTPUT_FILE, compression='gzip')

####
pd.concat([df.head(), df.tail()])

Queried data has 362762 rows, 362762 bars, 0 incompleted bars, from 2020-01-01 22:00:00+00:00 to 2020-12-31 21:59:00+00:00 US/Eastern.


,time,complete,volume,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2020-01-01 22:00:00+00:00,True,1.0,1.12083,1.12083,1.12083,1.12083,1.12165,1.12165,1.12165,1.12165
1,2020-01-01 22:01:00+00:00,True,1.0,1.12094,1.12094,1.12094,1.12094,1.12182,1.12182,1.12182,1.12182
2,2020-01-01 22:02:00+00:00,True,4.0,1.12094,1.12106,1.12093,1.12106,1.12182,1.12186,1.12182,1.12186
3,2020-01-01 22:03:00+00:00,True,3.0,1.12101,1.12101,1.12083,1.12083,1.12182,1.12182,1.12182,1.12182
4,2020-01-01 22:05:00+00:00,True,2.0,1.12087,1.12087,1.12083,1.12083,1.12187,1.12187,1.12183,1.12183
362757,2020-12-31 21:55:00+00:00,True,20.0,1.22177,1.22177,1.22167,1.22172,1.22192,1.22201,1.22191,1.22194
362758,2020-12-31 21:56:00+00:00,True,20.0,1.22165,1.22165,1.22157,1.22161,1.22195,1.22195,1.22180,1.22182
362759,2020-12-31 21:57:00+00:00,True,11.0,1.22155,1.22162,1.22145,1.22159,1.22183,1.22185,1.22173,1.22181
362760,2020-12-31 21:58:00+00:00,True,27.0,1.22149,1.22165,1.22128,1.22134,1.22186,1.22198,1.22178,1.22195
362761,2020-12-31 21:59:00+00:00,True,3.0,1.22130,1.22130,1.22121,1.22121,1.22191,1.22191,1.22190,1.22190
